In [1]:
import os
import numpy as np 
import sys

sys.path.append("..")
import tomondt
from tomondt import Operator
import pickle

In [13]:
times=  np.linspace(1, 100, 100)
gr, alpha = (1+np.sqrt(5))/2, np.radians(140)
angles = np.degrees(((times *gr*alpha)%alpha)-(alpha/2))

main_path = 'Y:\TimC\DIPSTER-PublicationData\Publication-Data'
v_path = os.path.join(main_path, 'Volumes')
ts_path = os.path.join(main_path, 'TiltSeries')

for item in os.listdir(v_path):
    if '.vmf' in item:
        if 'Cube' in item:
            full_path = os.path.join(v_path, item)
            base_filename, extension = os.path.splitext(full_path)
            vmf = tomondt.load_vmf(full_path)
            ts = tomondt.Operator('cupy', 0).fp(vmf,angles)
            with open(os.path.join(ts_path, base_filename+'-ts.pkl'), "wb") as pickleFile:
                pickle.dump(ts, pickleFile)




In [2]:
times=  np.linspace(1, 100, 100)
gr, alpha = (1+np.sqrt(5))/2, np.radians(140)
angles = np.degrees(((times *gr*alpha)%alpha)-(alpha/2))

main_path = 'Y:\TimC\DIPSTER-PublicationData\Publication-Data'
v_path = os.path.join(main_path, 'Reconstructions')
ts_path = os.path.join(main_path, 'TiltSeries')

for item in os.listdir(ts_path):
    if '.pkl' in item:
        full_path = os.path.join(ts_path, item)
        base_filename, extension = os.path.splitext(full_path)
        ts = pickle.load(open(full_path, 'rb'))
        algor = lambda x: tomondt.algorithms.sirt(x)
        new_path = os.path.join(v_path, base_filename+'-'+ str((np.degrees(alpha)/2).astype(int))+'-rec.vmf') 
        ts = tomondt.Operator('cupy', 0).bp(new_path, ts ,algor, 50)


KeyboardInterrupt: 

In [6]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

main_path = 'Y:\TimC\DIPSTER-PublicationData\Publication-Data'
v_path = os.path.join(main_path, 'Reconstructions')
ts_path = os.path.join(main_path, 'Volumes')

l_ssim = lambda x,y: ssim(x,y,data_range=1)



ref = tomondt.load_vmf(os.path.join(ts_path,'Cage-1.vmf'))
vol = tomondt.load_vmf(os.path.join(v_path,'Cage-1-ts-70-rec.vmf'))

qr =tomondt.QualityReport(os.path.join(v_path, 'test_qr.h5df'))
qr.add_test('SSIM',tomondt.qr.TestType.VolVol,l_ssim)
qr.add_testdata('TestSample', ref,S001=vol )

ref = tomondt.load_vmf(os.path.join(ts_path,'Cage-0.vmf'))
vol = tomondt.load_vmf(os.path.join(v_path,'Cage-0-ts-70-rec.vmf'))
qr.add_testdata('TestSample', ref,S000=vol )

qr = tomondt.Operator().evaluate(qr,0)

In [2]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import mrcz

main_path = 'Y:\TimC\DIPSTER-PublicationData\Publication-Data'
v_path = os.path.join(main_path, 'Reconstructions')
ts_path = os.path.join(main_path, 'Volumes')

ref = tomondt.load_vmf(os.path.join(ts_path,'Cage-1.vmf')).read_record(0).astype(np.float64)
ts_path_2 = os.path.join(main_path, 'TiltSeries')
ts = pickle.load(open(os.path.join(ts_path_2,'Cage-1-ts.pkl'), 'rb'))
angles = ts.angles
ts.data = tomondt.algorithms.forward_astra(ref,angles).astype(np.float64)
#ts.data = tomondt.utils.permute(ts.data,(0,1,2))
vol =  tomondt.algorithms.sirt_astra(ts,100)
print(ssim(ref,vol,data_range=1.1))
view = tomondt.Viewer()
mrcz.writeMRC(vol, os.path.join(v_path,'Cage-1-ts-70-rec.rec'))
view.add_image(vol)
view.add_image(ref)



Exception: Unable to initialize Projector2D.

In [11]:
mrcz.writeMRC(ref, os.path.join(v_path,'ref.rec'))

In [9]:
p = tomondt.Plots()
p.plot_qr(qr,3, True)
pickle.dump(qr, open(os.path.join(v_path, 'test_qr.pkl'), 'wb'))

PicklingError: Can't pickle <function <lambda> at 0x000001A7526F84A0>: attribute lookup <lambda> on __main__ failed